In [1]:
import pandas as pd
import geopandas as gpd
import os
#import maplib
import fiona
import csv

In [48]:
#read in GLiHT footprints
howland_fn = '/adapt/nobackup/people/zwwillia/dataCollections/GLiHT/Howland/Howland_footprints.gpkg'
serc_fn = '/adapt/nobackup/people/zwwillia/dataCollections/GLiHT/Serc/SERC_footprints.gpkg'

serc_gdb = gpd.read_file(serc_fn)
howland_gdb = gpd.read_file(howland_fn)
howland_gdb.head()

#read in SR-Lite footprints
srlite_fn = '/explore/nobackup/projects/ilab/projects/srlite/output/products/footprints/footprints_srlite-0.9.9.gpkg'
srlite_gdb = gpd.read_file(srlite_fn)
srlite_gdb.head()

print('SR Lite scenes/projection:')
print(len(srlite_gdb))
print(srlite_gdb.crs)
print()
print('GLiHT Howland scenes/projection:')
print(len(howland_gdb))
print(howland_gdb.crs)
print()
print('GLiHT SERC scenes/projection:')
print(len(serc_gdb))
print(serc_gdb.crs)

SR Lite scenes/projection:
182
epsg:4326

GLiHT Howland scenes/projection:
266
epsg:32619

GLiHT SERC scenes/projection:
28
epsg:32618


In [54]:
#if mismatched crs, reproject
howland_gdb = howland_gdb.to_crs(srlite_gdb.crs)
print('Howland Projection:')
print(howland_gdb.crs)
serc_gdb = serc_gdb.to_crs(srlite_gdb.crs)
print('SERC Projection:')
print(serc_gdb.crs)

Howland Projection:
EPSG:4326
SERC Projection:
EPSG:4326


In [67]:
#find matches
#srlite_gdb['savedindex'] = srlite_gdb.index
intersect_howland = gpd.overlay(howland_gdb,srlite_gdb,how='intersection')

print('Scene matches:')
print(len(intersect_howland))

keep_col_list = ['location', 'path', 'file','area_km2']
#rm_col_list = intersect.columns.difference(keep_col_list).tolist()
#intersect = intersect.drop(rm_col_list, axis=1)
intersect.head()

Scene matches:
224


,location,geometry,index_right,footprint_name,path,file,area_km2,area_ha
172,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,"POLYGON ((-68.78141 45.14496, -68.78075 45.144...",132,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160117_M1BS_10300100505E7E00-sr-02m.tif,312.248316,31224.8316
173,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,"POLYGON ((-68.78141 45.14496, -68.78075 45.144...",132,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160117_M1BS_10300100505E7E00-sr-02m.tif,312.248316,31224.8316
174,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,"POLYGON ((-68.78849 45.15019, -68.78071 45.150...",132,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160117_M1BS_10300100505E7E00-sr-02m.tif,312.248316,31224.8316
175,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,"POLYGON ((-68.78849 45.15019, -68.78071 45.150...",132,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160117_M1BS_10300100505E7E00-sr-02m.tif,312.248316,31224.8316
178,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,"POLYGON ((-68.78076 45.14715, -68.72007 45.147...",132,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160117_M1BS_10300100505E7E00-sr-02m.tif,312.248316,31224.8316


In [73]:
intersect_howland = gpd.overlay(howland_gdb,srlite_gdb,how='intersection')
print(len(intersect_howland))
intersect_howland.head()

224


,location,footprint_name,path,file,area_km2,area_ha,geometry
0,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160114_M1BS_1030010050497600-sr-02m.tif,396.972864,39697.2864,"POLYGON ((-68.78075 45.14496, -68.78075 45.143..."
1,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160114_M1BS_1030010050497600-sr-02m.tif,396.972864,39697.2864,"POLYGON ((-68.78075 45.14496, -68.78075 45.143..."
2,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160114_M1BS_1030010050497600-sr-02m.tif,396.972864,39697.2864,"POLYGON ((-68.78071 45.15018, -68.78073 45.144..."
3,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160114_M1BS_1030010050497600-sr-02m.tif,396.972864,39697.2864,"POLYGON ((-68.78071 45.15018, -68.78073 45.144..."
4,./AMIGACarb_Bangor_FIA_Jun2012/AMIGACarb_Bango...,None,/adapt/nobackup/projects/ilab/data/srlite/prod...,WV02_20160114_M1BS_1030010050497600-sr-02m.tif,396.972864,39697.2864,"POLYGON ((-68.72007 45.14702, -68.72024 45.113..."


In [75]:
intersect_howland.to_file('/adapt/nobackup/people/zwwillia/dataCollections/GLiHT/Howland/SR_lite_intersect.gpkg',driver='GPKG')